In [20]:
import pandas as pd

content = pd.read_json('content.jsonl', lines=True)
rating = pd.read_json('ratings.jsonl', lines=True)
targets = pd.read_csv('targets.csv')

In [16]:
# content['Genre'] = content['Genre'].apply(lambda x: x.split(','))
# content = content.explode('Genre')
# content = pd.get_dummies(content, columns=['Genre'])
# content.head()

,ItemId,Title,Year,Rated,Released,Runtime,Director,Writer,Actors,Plot,...,Genre_News,Genre_Reality-TV,Genre_Romance,Genre_Sci-Fi,Genre_Short,Genre_Sport,Genre_Talk-Show,Genre_Thriller,Genre_War,Genre_Western
0,c9f0f895fb,Edison Kinetoscopic Record of a Sneeze,1894,N/A,09 Jan 1894,1 min,William K.L. Dickson,N/A,Fred Ott,A man (Edison's assistant) takes a pinch of sn...,...,False,False,False,False,False,False,False,False,False,False
0,c9f0f895fb,Edison Kinetoscopic Record of a Sneeze,1894,N/A,09 Jan 1894,1 min,William K.L. Dickson,N/A,Fred Ott,A man (Edison's assistant) takes a pinch of sn...,...,False,False,False,False,False,False,False,False,False,False
1,d3d9446802,Leaving the Factory,1895,Not Rated,22 Mar 1895,1 min,Louis Lumière,N/A,N/A,A man opens the big gates to the Lumière facto...,...,False,False,False,False,False,False,False,False,False,False
1,d3d9446802,Leaving the Factory,1895,Not Rated,22 Mar 1895,1 min,Louis Lumière,N/A,N/A,A man opens the big gates to the Lumière facto...,...,False,False,False,False,False,False,False,False,False,False
2,c20ad4d76f,The Arrival of a Train,1896,Not Rated,25 Jan 1896,1 min,"Auguste Lumière, Louis Lumière",N/A,"Madeleine Koehler, Marcel Koehler, Mrs. August...",A group of people are standing in a straight l...,...,False,False,False,False,False,False,False,False,False,False


In [35]:
#one hot encoding de generoS
genres = content[['ItemId', 'Genre']]
genres['Genre'] = genres['Genre'].apply(lambda x: x.split(','))
genres = genres.explode('Genre')
genres = pd.get_dummies(genres, columns=['Genre'])
genres.head()

/var/folders/m8/tpsps8yn38xg64w47wpvv3nr0000gn/T/ipykernel_45361/3643481053.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres['Genre'] = genres['Genre'].apply(lambda x: x.split(','))


,ItemId,Genre_ Action,Genre_ Adult,Genre_ Adventure,Genre_ Animation,Genre_ Biography,Genre_ Comedy,Genre_ Crime,Genre_ Documentary,Genre_ Drama,...,Genre_News,Genre_Reality-TV,Genre_Romance,Genre_Sci-Fi,Genre_Short,Genre_Sport,Genre_Talk-Show,Genre_Thriller,Genre_War,Genre_Western
0,c9f0f895fb,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,c9f0f895fb,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,d3d9446802,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,d3d9446802,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,c20ad4d76f,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [34]:
user_profile =  pd.merge(rating, genres, on='ItemId')

# Assumindo que 'user_profile' já é o resultado do merge entre 'rating' e 'genres'.
# E que as colunas de gênero no DataFrame 'genres' são algo como 'Genre_Action', 'Genre_Drama', etc.

# Identifique as colunas de gênero
genre_columns = [col for col in genres.columns if 'Genre_' in col]

# Calcule a soma de ratings por gênero para cada usuário
sums = user_profile.groupby('UserId')[genre_columns].sum()

# Calcule a contagem de ratings por gênero para cada usuário
counts = user_profile.groupby('UserId')[genre_columns].count()

# Crie um novo DataFrame para as médias
averages = sums / counts

# Substitua NaNs por zeros
averages.fillna(0, inplace=True)

# Converta o DataFrame de médias em um dicionário aninhado
# Onde a chave externa é o UserID e a chave interna são os gêneros
user_genre_averages_dict = averages.to_dict(orient='index')

# Agora `user_genre_averages_dict` é um dicionário com UserIDs como chaves
# E cada valor é outro dicionário com gêneros como chaves e a média de ratings como valores

#imprimir o dicionario do user 0004d0b59e
print(user_genre_averages_dict['0004d0b59e'])


{'Genre_ Action': 0.0, 'Genre_ Adult': 0.0, 'Genre_ Adventure': 0.0, 'Genre_ Animation': 0.0, 'Genre_ Biography': 0.3333333333333333, 'Genre_ Comedy': 0.0, 'Genre_ Crime': 0.0, 'Genre_ Documentary': 0.0, 'Genre_ Drama': 0.3333333333333333, 'Genre_ Family': 0.0, 'Genre_ Fantasy': 0.0, 'Genre_ Film-Noir': 0.0, 'Genre_ Game-Show': 0.0, 'Genre_ History': 0.0, 'Genre_ Horror': 0.0, 'Genre_ Music': 0.0, 'Genre_ Musical': 0.0, 'Genre_ Mystery': 0.0, 'Genre_ News': 0.0, 'Genre_ Reality-TV': 0.0, 'Genre_ Romance': 0.0, 'Genre_ Sci-Fi': 0.0, 'Genre_ Short': 0.0, 'Genre_ Sport': 0.0, 'Genre_ Talk-Show': 0.0, 'Genre_ Thriller': 0.0, 'Genre_ War': 0.0, 'Genre_ Western': 0.0, 'Genre_Action': 0.3333333333333333, 'Genre_Adult': 0.0, 'Genre_Adventure': 0.0, 'Genre_Animation': 0.0, 'Genre_Biography': 0.0, 'Genre_Comedy': 0.0, 'Genre_Crime': 0.0, 'Genre_Documentary': 0.0, 'Genre_Drama': 0.0, 'Genre_Family': 0.0, 'Genre_Fantasy': 0.0, 'Genre_Film-Noir': 0.0, 'Genre_History': 0.0, 'Genre_Horror': 0.0, 'Gen

KeyError: '0004d0b59e'